**Question 1:**

In [30]:
import datetime
import pandas as pd
import re

Below is the designed date function which can handle several date formats and also handle invalid dates which will be marked as NaT.

In [86]:
def datetime_detectandconvert(df):
  drop_list=[] #This list is used collect the column names which are converted to the date format
  
  #In first handling we will be using the existing to_datetime for column conversion to datetime
  for col in df.columns:
    if df[col].dtype == 'object': #Checking if the column type is object or not for date
        try:
            df[col] = pd.to_datetime(df[col]) #converion to datetime
        except ValueError:
            pass

  #In this attempt we will be check the column which the above part ignored as the above method cannot handle various format
  for col in df:
    if df[col].dtypes == object:

      # Now we are checking the column if it consist date or not using regular expression, the regular expression can be updated to match according to requirement 
      if df[col].str.match(r"^([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])$").any()==True or df[col].str.match(r"^([0-9][0-9]|19[0-9][0-9]|20[0-9][0-9])(\.|-|/)([1-9]|0[1-9]|1[0-2])(\.|-|/)([1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])$").any()==True:
        k = 'New'+col # for new column
        drop_list.append(col)
        df[k] = pd.to_datetime(df[col] , format='%Y-%m-%d', errors='coerce') # Converting the value present in original column containg date and storing it in the new column
        mask = df[k].isnull() # check how many null value are their which were left due to not matchinf the format
        
        # Similar process like above are done for the various format
        df.loc[mask, k] = pd.to_datetime(df[mask][col], format='%d%m%Y',errors='coerce')
        mask1 = df[k].isnull()
        df.loc[mask1, k] = pd.to_datetime(df[mask1][col], format='%Y-%d-%m',errors='coerce')
        mask2 = df[k].isnull()
        df.loc[mask2, k] = pd.to_datetime(df[mask2][col], format='%d-%m-%Y',errors='coerce')
    else:
        continue
  
  df.drop(drop_list, axis=1, inplace=True) # dropping the original column which were converted

  # Now we are taking date columns and are making new columns which are difference between the existing columns taking 2 at a time
  for col in df:
    if df[col].dtype == 'datetime64[ns]':
      for col1 in df:
        if df[col1].dtype == 'datetime64[ns]' and col1 != col:
          k = col + ' - ' + col1
          df[k]=df[col] - df[col1]
        else:
          continue
    else:
      continue


Case 1 :

In this case I have designed as small dataframe which consist of several date format, Which are the used to check the function if it can handle several date formats.

In [80]:
df=pd.DataFrame({"status": ['Opened','Opened','Closed','Opened','Opened','Closed'],
                 "A":["2019-01-01","08012019", "14-13-2019", "20190703","2019/07/03","07-03-2019"],
                 "B":["2019-03-02","08012019", "2019-08-01", "20190703","2019/07/03","07-03-2019"] })
df.dtypes

status    object
A         object
B         object
dtype: object

In [87]:
# Calling the function and checking the dtypes
datetime_detectandconvert(df)
df.dtypes

status                  object
NewA            datetime64[ns]
NewB            datetime64[ns]
NewA - NewB    timedelta64[ns]
NewB - NewA    timedelta64[ns]
dtype: object

In [83]:
df.head(6) 

,status,A,B
0,Opened,2019-01-01,2019-03-02
1,Opened,08012019,08012019
2,Closed,14-13-2019,2019-08-01
3,Opened,20190703,20190703
4,Opened,2019/07/03,2019/07/03
5,Closed,07-03-2019,07-03-2019


In [88]:
url = 'http://bit.ly/uforeports'
  
# read csv file 
df1 = pd.read_csv(url) # Reading the dataset which have column in datetime format consisting both date and time within it and solely for checking it only            
df1.head() 

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


In [89]:
df1.dtypes

City               object
Colors Reported    object
Shape Reported     object
State              object
Time               object
dtype: object

In [90]:
datetime_detectandconvert(df1) # calling the function and checking if it is converting
df1.dtypes

City                       object
Colors Reported            object
Shape Reported             object
State                      object
Time               datetime64[ns]
dtype: object

In [91]:
df1.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00


**Question 2:**

In [1]:
import datetime
import pandas as pd
import re

The solution for question 2 is incomplete, I have propsed what steps can be taken further for solving this problem in the last cell.

In [7]:
# Reading the dataset

path = '/content/drive/My Drive/Dataset.xls'
df4 = pd.read_excel(path) 

In [8]:
# Cheking the dataset

df4.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [9]:
df4.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

Checking the pearson coefficient between the columns

In [11]:
dfc = df4.corr(method ='pearson').abs()
dfc

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,1.000000,0.026179,0.018497,0.039177,0.029079,0.018678,0.030575,0.011215,0.018494,0.002735,0.022199,0.020270,0.019389,0.017982,0.024354,0.040351,0.016705,0.016730,0.009742,0.008406,0.039151,0.007793,0.000652,0.003000,0.013952
LIMIT_BAL,0.026179,1.000000,0.024755,0.219161,0.108139,0.144713,0.271214,0.296382,0.286123,0.267460,0.249411,0.235195,0.285430,0.278314,0.283236,0.293988,0.295562,0.290389,0.195236,0.178408,0.210167,0.203242,0.217202,0.219595,0.153520
SEX,0.018497,0.024755,1.000000,0.014232,0.031389,0.090874,0.057643,0.070771,0.066096,0.060173,0.055064,0.044008,0.033642,0.031183,0.024563,0.021880,0.017005,0.016733,0.000242,0.001391,0.008597,0.002229,0.001667,0.002766,0.039961
EDUCATION,0.039177,0.219161,0.014232,1.000000,0.143464,0.175061,0.105364,0.121566,0.114025,0.108793,0.097520,0.082316,0.023581,0.018749,0.013002,0.000451,0.007567,0.009099,0.037456,0.030038,0.039943,0.038218,0.040358,0.037200,0.028006
MARRIAGE,0.029079,0.108139,0.031389,0.143464,1.000000,0.414170,0.019917,0.024199,0.032688,0.033122,0.035629,0.034345,0.023472,0.021602,0.024909,0.023344,0.025393,0.021207,0.005979,0.008093,0.003541,0.012659,0.001205,0.006641,0.024339
AGE,0.018678,0.144713,0.090874,0.175061,0.414170,1.000000,0.039447,0.050148,0.053048,0.049722,0.053826,0.048773,0.056239,0.054283,0.053710,0.051353,0.049345,0.047613,0.026147,0.021785,0.029247,0.021379,0.022850,0.019478,0.013890
PAY_0,0.030575,0.271214,0.057643,0.105364,0.019917,0.039447,1.000000,0.672164,0.574245,0.538841,0.509426,0.474553,0.187068,0.189859,0.179785,0.179125,0.180635,0.176980,0.079269,0.070101,0.070561,0.064005,0.058190,0.058673,0.324794
PAY_2,0.011215,0.296382,0.070771,0.121566,0.024199,0.050148,0.672164,1.000000,0.766552,0.662067,0.622780,0.575501,0.234887,0.235257,0.224146,0.222237,0.221348,0.219403,0.080701,0.058990,0.055901,0.046858,0.037093,0.036500,0.263551
PAY_3,0.018494,0.286123,0.066096,0.114025,0.032688,0.053048,0.574245,0.766552,1.000000,0.777359,0.686775,0.632684,0.208473,0.237295,0.227494,0.227202,0.225145,0.222327,0.001295,0.066793,0.053311,0.046067,0.035863,0.035861,0.235253
PAY_4,0.002735,0.267460,0.060173,0.108793,0.033122,0.049722,0.538841,0.662067,0.777359,1.000000,0.819835,0.716449,0.202812,0.225816,0.244983,0.245917,0.242902,0.239154,0.009362,0.001944,0.069235,0.043461,0.033590,0.026565,0.216614


Taking upper half of the pearson coeffient table for further proceess

In [13]:
import numpy as np

upper = dfc.where(np.triu(np.ones(dfc.shape), k=1).astype(np.bool))


In [14]:
upper

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,NaN,0.026179,0.018497,0.039177,0.029079,0.018678,0.030575,0.011215,0.018494,0.002735,0.022199,0.020270,0.019389,0.017982,0.024354,0.040351,0.016705,0.016730,0.009742,0.008406,0.039151,0.007793,0.000652,0.003000,0.013952
LIMIT_BAL,NaN,NaN,0.024755,0.219161,0.108139,0.144713,0.271214,0.296382,0.286123,0.267460,0.249411,0.235195,0.285430,0.278314,0.283236,0.293988,0.295562,0.290389,0.195236,0.178408,0.210167,0.203242,0.217202,0.219595,0.153520
SEX,NaN,NaN,NaN,0.014232,0.031389,0.090874,0.057643,0.070771,0.066096,0.060173,0.055064,0.044008,0.033642,0.031183,0.024563,0.021880,0.017005,0.016733,0.000242,0.001391,0.008597,0.002229,0.001667,0.002766,0.039961
EDUCATION,NaN,NaN,NaN,NaN,0.143464,0.175061,0.105364,0.121566,0.114025,0.108793,0.097520,0.082316,0.023581,0.018749,0.013002,0.000451,0.007567,0.009099,0.037456,0.030038,0.039943,0.038218,0.040358,0.037200,0.028006
MARRIAGE,NaN,NaN,NaN,NaN,NaN,0.414170,0.019917,0.024199,0.032688,0.033122,0.035629,0.034345,0.023472,0.021602,0.024909,0.023344,0.025393,0.021207,0.005979,0.008093,0.003541,0.012659,0.001205,0.006641,0.024339
AGE,NaN,NaN,NaN,NaN,NaN,NaN,0.039447,0.050148,0.053048,0.049722,0.053826,0.048773,0.056239,0.054283,0.053710,0.051353,0.049345,0.047613,0.026147,0.021785,0.029247,0.021379,0.022850,0.019478,0.013890
PAY_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.672164,0.574245,0.538841,0.509426,0.474553,0.187068,0.189859,0.179785,0.179125,0.180635,0.176980,0.079269,0.070101,0.070561,0.064005,0.058190,0.058673,0.324794
PAY_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.766552,0.662067,0.622780,0.575501,0.234887,0.235257,0.224146,0.222237,0.221348,0.219403,0.080701,0.058990,0.055901,0.046858,0.037093,0.036500,0.263551
PAY_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.777359,0.686775,0.632684,0.208473,0.237295,0.227494,0.227202,0.225145,0.222327,0.001295,0.066793,0.053311,0.046067,0.035863,0.035861,0.235253
PAY_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.819835,0.716449,0.202812,0.225816,0.244983,0.245917,0.242902,0.239154,0.009362,0.001944,0.069235,0.043461,0.033590,0.026565,0.216614


Check which column has person coeffiecient above 0.85

In [15]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop

['BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']

For further process my approach will be like : 


*   I will check which of them between each of them have more corelation
*   The for further inspection will use Recursive Feature Elimination for better understanding
*   Check other feature selection method which will provide a better understanding
*   As only checking pearson correaltion may not always be a better way to understand the data and the effect of each column on the result











Thank you for this opportunity , The test was very informative to me, It helped me to enhace my skill.